# WFSS trace plotting notebook

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook visualizes the spectral traces for WFSS observations for a given orientation of the telescope (this is an early version, so the tools for WFSS trace visualization may evolve over time).

This notebook was originally designed for observations that use target acquisition on one star and then slew to another. WFSS does not do this, so users should use the same coordinates for their ACQ target as for their SCI target. 

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirment Offset" X and Y values needed to place the SCI star at the reference position, and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

- With installing the module:
  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .`
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.
- Without installing the module:
  1. Copy this notebook and `utils.py` to your working directory.
  2. In the cell below, change `from all_jwst_offset_ta import utils` to `import utils`
  3. Open the notebook as you normally would. The following python modules must be installed in the working enviroment:
     -  `ipywidgets`
     - `numpy`
     - `matplotlib`
     - `astropy`
     - `pysiaf`

In [1]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [3]:
# for your system, choose an appropriate plotting backend to make pop-up window plots. 
# Currently, inline plots do not work.
%matplotlib osx

In [6]:
from all_jwst_offset_ta import utils
# if the library is not installed replace the line above with the line below 
# when `utils.py` is in the same folder as the notebook
# import utils

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-068 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-071
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


## Usage

1. Run the cell below, which will generate an interactive widget in the output space.
2. Select your instrument and science aperture from the drop-down menus.
   - The science aperture names can be a bit opaque, but you should be able to guess which one corresponds to your observation.
   - If you have access to data that uses the subarray of interest, you can check the header for the "APERNAME" there and match it.
3. Enter the RA and Dec coordinates of your acquisition and science targets.
   - the frame must be 'ICRS'
   - the units must be in decimal degrees
4. Enter the PA angle of the V3 axis of the telescope at the instrument reference point.
   - In APT, this corresponds to the "Specify PA range using V3PA" option under Special Requirements.
   - This angle is given in data headers as "ROLL_REF"
5. If you wish to know the positions of other stars in the field of view, you can enter them in the "Other stars" box
   - The format is `label: (ra, dec)` where `ra` and `dec` are given in decimal degrees
   - There is currently no error checking for formatting; you will just make it crash/get a wrong answer
   - For multi-line entries, you may need to type them elsewhere and they copy-paste the list into the box
7. Press "Compute offset" to get the offsets to enter into APT (you may need to press it twice).
   - You will also get the positions in IDL coordinates of all the stars before and after the slew.
   - For same-target TA, a hack is to enter the same position for the ACQ and SCI target

Copy the lines below and paste them into the `Other stars` box, as an example:
```
a: (1.0, 2.0)
b: (2.0, 3.0)
c: (3.0, 4.0)
```

In [7]:
# create the ComputeOffsets object and start the UI with default initialization.
co1 = utils.ComputeOffsets()
# to display the user interface, just do this:
co1.ui

You can also access the IDL coordinates from the object to use them programatically:

(note: If the cell below throws an error, it's because you have to press the "Compute offset" button)

In [8]:
co1.idl_coords_after_ta

{'ACQ': array([-2.84203838e-14,  2.74500713e-16]),
 'SCI': array([-2.84203838e-14,  2.74500713e-16])}

In [9]:
co1.idl_coords_after_slew

{'ACQ': array([0., 0.]), 'SCI': array([0., 0.])}

## Initialization from a configuration dictionary

Alternatively, one can provide a dictionary to pre-populate the fields of the interface. This is useful for repeatability/record keeping purposes. A non-empty dictionary will automatically trigger the output calculations, filling in default values for missing entries.

As an example, let's use MIRI's 15.5 um coronagraph to do TA on a star at (0, 0) and then slew to another star at (50 arcsec, 50 arcsec), with the telescope oriented at a V3_PA angle of 180 deg. There are three other stars of interest, labeled a and b, located at coordinates (60 arcsec, 40 arcsec), and (40 arcsec, 60 arcsec), respectively.

**<span style="color:red">WARNING</span>**: for each set of calculations, create a new "ComputeOffsets" object. Calling the same UI object again changes its values globally.

In [10]:
from astropy import units
initial_values={
    'instr': 'miri',
    'sci_aper': 'mirim_illum', 
    'pa': 180.,
    'acq_ra': (50*units.arcsec).to("deg").value, 'acq_dec': (50*units.arcsec).to("deg").value,
    'sci_ra': (50*units.arcsec).to("deg").value, 'sci_dec': (50*units.arcsec).to("deg").value,
}
# add a multi-line string of the other stars
initial_values['other_stars'] = """
a: (0.017, 0.011)
b: (0.011, 0.017)
""".strip()

In [11]:
co = utils.ComputeOffsets(initial_values=initial_values)
co.ui

## An example of a real field

The source coordinates below are taken from a real commissioning activity, where we needed to check that the traces wouldn't overlap.

In [15]:
# some 
sources = {
    'SCI': utils.SkyCoord("05 24 20.7552 -70 05 1.60", frame='icrs', unit=("hourangle","degree")),
    'a': utils.SkyCoord("05 24 26.33969 -70 05 22.3545", frame='icrs', unit=("hourangle","degree")),
    'b': utils.SkyCoord("05 24 28.67861 -70 05 24.4484", frame='icrs', unit=("hourangle","degree")),
    'c': utils.SkyCoord("05 24 36.22460 -70 05 28.1876", frame='icrs', unit=("hourangle","degree")),
    'd': utils.SkyCoord("05 24 25.608 -70 05 01.66", frame='icrs', unit=("hourangle","degree")),
}

In [16]:
# format the sources for easy copy-pasting
for s, c in sources.items():
    print(f"{s}: ({c.ra.deg}, {c.dec.deg})")

SCI: (81.08648, -70.08377777777777)
a: (81.10974870833334, -70.08954291666666)
b: (81.11949420833332, -70.09012455555555)
c: (81.15093583333332, -70.09116322222222)
d: (81.1067, -70.08379444444444)


In [17]:
from astropy import units
initial_values={
    'instr': 'miri',
    'sci_aper': 'mirim_illum', 
    'pa': 290.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = """
a: (81.10974870833334, -70.08954291666666)
b: (81.11949420833332, -70.09012455555555)
c: (81.15093583333332, -70.09116322222222)
d: (81.1067, -70.08379444444444)
""".strip()

In [18]:
co1 = utils.ComputeOffsets(initial_values=initial_values)
co1.ui

## Plot WFSS traces

To plot the WFSS traces, use the function below.

In [19]:
import matplotlib as mpl
from matplotlib import pyplot as plt

def plot_wfss_traces(
    idl_coords : dict,
    aper : utils.pysiaf.JwstAperture,
    ax : mpl.axes.Axes = None,
    title : str = '',
    show_mirim_illum = True,
    plot_full : bool = False
):
    """
    Compute the idl coordinates overlaid on the aperture of interest. Add the WFSS traces as well.
    idl_coords : dict[label, tuple[ra, dec]]
      usually, one of utils.ComputeOffsets().idl_coords_after_ta or
      utils.ComputeOffsets().idl_coords_after_slew
    aper : usually the return value of utils.ComputeOffsets().get_aper()
    ax : axis to plot on
    title : title for the axis
    show_aper_title : if True, display the text MIRIM_ILLUM
    plot_full : if True, also plot the FULL detector array
    """
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
    ax.set_title(title)
    trace_up = 100 * aper.YSciScale
    trace_down = 300 * aper.YSciScale
   
    for i, (k, coord) in enumerate(idl_coords.items()):
        width = 1
        height = trace_up + trace_down
        ll = (coord[0]-width/2, coord[1]-trace_down)
        trace = mpl.patches.Rectangle(ll, width, height, facecolor=f'C0', alpha=0.5)
        ax.add_patch(trace)
        ax.scatter(*coord, c=f'C{i}', marker='x',label=k)
    aper.plot(ax=ax, fill=False, mark_ref=False, frame='idl', label=True, zorder=-1)

    if plot_full:
        full_aper = utils.Siaf("MIRI")['MIRIM_FULL']
        # conver the corners to ILLUM IDL
        corners = aper.convert(*full_aper.corners("tel"), from_frame="tel", to_frame='idl')
        full_rect = mpl.patches.Rectangle(
            xy = np.min(corners, axis=1),
            width = np.diff(corners[0]).max(),
            height = np.diff(corners[1]).max(),
            fill=False, ec='gray'
        )
        ax.add_patch(full_rect)
    ax.legend(loc=(1.05, 0.3), title='Sources')
    return fig

In [20]:
coords_to_plot = co1.idl_coords_after_slew.copy()
# remove the ACQ star from the plot
coords_to_plot.pop("ACQ")
coords_to_plot["VI"] = coords_to_plot.pop("SCI")
fig = plot_wfss_traces(
    coords_to_plot, 
    aper = co1.get_aper(),
    title=f"PA = {co1._initial_values['pa']}",
    show_mirim_illum=True
)